In [10]:
import os, itertools, random, imageio, sklearn
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import matplotlib.pyplot as plt

In [11]:
faces_folder = "faces"
pickle_file = "faces.pickle"
trained_models_folder = "./tf_trained/"

pixel_depth = 255.0
image_size = 128
num_labels = 2
input_size = image_size * image_size * 2

In [12]:
def accuracy(predictions, labels):
  compare_elements = np.argmax(predictions, 1) == np.argmax(labels, 1)
  return (100.0 * np.sum(compare_elements) / predictions.shape[0])

def makedir(path):
  if not os.path.exists(path):
    os.makedirs(path)
    
def accuracy_averaged(steps, accuracy_over_time, every_index=10):
  new_accuracy = np.mean(np.array(accuracy_over_time).reshape(-1, every_index), axis=1)
  new_steps = steps[0::every_index]
  return new_steps, new_accuracy

In [13]:
n_bytes = 2**31
max_bytes = 2**31 - 1
bytes_in = bytearray(0)

pickle_file_size = os.path.getsize(pickle_file)
with open(pickle_file, 'rb') as f_in:
  for _ in range(0, pickle_file_size, max_bytes):
    bytes_in += f_in.read(max_bytes)
save = pickle.loads(bytes_in)

train_dataset = save['train_dataset']
train_labels = save['train_labels']
valid_dataset = save['valid_dataset']
valid_labels = save['valid_labels']
test_dataset = save['test_dataset']
test_labels = save['test_labels']
del save  # hint to help gc free up memory

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (23014, 32768) (23014, 2)
Validation set (1275, 32768) (1275, 2)
Test set (1275, 32768) (1275, 2)


In [14]:
# shuffleing the data
train_dataset, train_labels = sklearn.utils.shuffle(train_dataset, train_labels)
valid_dataset, valid_labels = sklearn.utils.shuffle(valid_dataset, valid_labels)
test_dataset, test_labels = sklearn.utils.shuffle(test_dataset, test_labels)
print("done shuffle")

done shuffle


In [15]:
batch_size = 128
epochs = 100

layer_sizes = {
  "1": 2048,
  "2": 1024,
  "3": 64
}

dropouts = {
  "1": 0.5,
  "2": 0.5,
  "3": 0.5,
}

model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(layer_sizes["1"], activation=tf.nn.relu, input_shape=(32768,), name="input"))
model.add(tf.keras.layers.Dropout(dropouts["1"],  name="dropout_input"))

model.add(tf.keras.layers.Dense(layer_sizes["2"], activation=tf.nn.relu, name="layer_1"))
model.add(tf.keras.layers.Dropout(dropouts["2"], name="dropout_layer_1"))

model.add(tf.keras.layers.Dense(layer_sizes["3"], activation=tf.nn.relu, name="layer_2"))
model.add(tf.keras.layers.Dropout(dropouts["3"], name="dropout_layer_2"))

model.add(tf.keras.layers.Dense(num_labels, activation=tf.nn.softmax, name="output"))

sgd = tf.keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(train_dataset, train_labels,
          epochs=epochs,
          batch_size=batch_size)

score = model.evaluate(test_dataset, test_labels, batch_size=batch_size)
print("Score", score)

model.save("same_people_detector_v1")

Epoch 1/100
23014/23014 [==============================] - 331s 14ms/step - loss: 0.7042 - acc: 0.5089
Epoch 2/100
23014/23014 [==============================] - 262s 11ms/step - loss: 0.6917 - acc: 0.5172
Epoch 3/100
23014/23014 [==============================] - 241s 10ms/step - loss: 0.6906 - acc: 0.5194
Epoch 4/100
23014/23014 [==============================] - 282s 12ms/step - loss: 0.6884 - acc: 0.5298
Epoch 5/100
23014/23014 [==============================] - 244s 11ms/step - loss: 0.6881 - acc: 0.5302
Epoch 6/100
23014/23014 [==============================] - 226s 10ms/step - loss: 0.6866 - acc: 0.5371
Epoch 7/100
23014/23014 [==============================] - 242s 11ms/step - loss: 0.6839 - acc: 0.5408
Epoch 8/100
23014/23014 [==============================] - 202s 9ms/step - loss: 0.6835 - acc: 0.5471
Epoch 9/100
23014/23014 [==============================] - 192s 8ms/step - loss: 0.6814 - acc: 0.5466
Epoch 10/100
23014/23014 [==============================] - 204s 9ms/step -

23014/23014 [==============================] - 232s 10ms/step - loss: 0.3043 - acc: 0.8812
Epoch 81/100
23014/23014 [==============================] - 191s 8ms/step - loss: 0.3069 - acc: 0.8807
Epoch 82/100
23014/23014 [==============================] - 179s 8ms/step - loss: 0.3051 - acc: 0.8812
Epoch 83/100
23014/23014 [==============================] - 180s 8ms/step - loss: 0.3030 - acc: 0.8821
Epoch 84/100
23014/23014 [==============================] - 179s 8ms/step - loss: 0.3047 - acc: 0.8826
Epoch 85/100
23014/23014 [==============================] - 181s 8ms/step - loss: 0.3128 - acc: 0.8762
Epoch 86/100
23014/23014 [==============================] - 182s 8ms/step - loss: 0.3026 - acc: 0.8826
Epoch 87/100
23014/23014 [==============================] - 181s 8ms/step - loss: 0.3079 - acc: 0.8802
Epoch 88/100
23014/23014 [==============================] - 180s 8ms/step - loss: 0.3077 - acc: 0.8805
Epoch 89/100
23014/23014 [==============================] - 180s 8ms/step - loss: 0.3